# Neural Networks 2 - Optimization, Backpropagation and Conclusion
<hr style="height:5px;border-width:2;color:gray">

## 0. Review

In this 2$^{nd}$ part of Neural Networks we'll look at the optimization function that we'll be using for a Neural Network. We'll also look at Backpropagation, an algorithm that revolutionalized parameter learning in Neural Networks. After that we'll take a look at a modified Gradient Descent algorithm called Mini-batch Gradient Descent and then briefly look at multiclass classification. Finally we'll sum up Neural Networks and take a look at the workflow of a Neural Network model. So this is going to be a long notebook but you're going to learn a lot and by the end of this notebook you'll have successfully unmasked this black box called Neural Networks. So, before we dive into all these new concepts, let's take a brief look at Part 1 and note the important points to remember.

<ul>
    <li>We first saw the general architecture of a Neural Network involving input layer, hidden layers and ouput layers. Each layer consists of a set of neurons called units which is just a single logistic unit. <b>Remember that an $L$ layered Neural network has $(L-1)$ hidden layers and $1$ output layer. Input layer is the $0^{th}$ layer</b>.</li>
    <li>
        Then we looked at Forward Propagation. Although the equations were given for a 2 layer network, it can be generalized to L layers as follows.<br>
        The forward propagation equations for the $l^{th}$ layer are given by,<br><br>
        $$\boxed{\begin{align}
        Z^{[l]} &= W^{[l]}.A^{[l-1]} + b^{[l]} \\\\
        A^{[l]} &= g^{[l]}(Z^{[l]})
        \end{align}}$$
        where, $g^{[l]}(.)$ is the activation function of $l^{th}$ layer.
    </li>
    <li> We then learnt initialization of parameters (<b>Randomly for weights and zeros for biases</b>). We also looked at the shapes of the parameter matrices.<br>
        If the $[l-1]^{th}$ layer has $n^{[l-1]}$ units and $l^{th}$ layer has $n^{[l]}$ units then, <br>
        <ul>
            <li>$W^{[l]}$ has shape $(n^{[l]} \times n^{[l-1]})$.</li>
            <li>$b^{[l]}$ has shape $(n^{[l]} \times 1)$.</li>
        </ul>
    </li>
    <li>Later we looked at the different kinds of non-linear functions called <b>activation functions</b> like ReLU, Leaky ReLU, $\tanh$ and saw their advantages and disadvantages.</li>
    <li>Note that for a quantity X, <br>
        $X^{(i)[l]}_j$ means that<br>
        <ul>
            <li>It is in the $[l]^{th}$ layer.</li>
            <li>It is in the $j^{th}$ unit of that layer.</li>
            <li>It is part of the $(i)^{th}$ training example.</li>
        </ul>
    </li>
</ul>
<br>
Now that we are done with the review let's dive right in.

<hr style="height:2px;border-width:2;color:gray">

## 1. Optimization Function

After randomly initializing weights and biases, you have performed a forward pass through the neural network and got an output , a value between 0 and 1. But how do you measure how good or how bad of an output this is? That’s where our optimization function steps in.<br>
As we have already seen in Linear Regression and Logistic Regression, the optimization function is the function which we need to minimize or maximize. In our case we need to minimize the cost function.

#### 1.1. The Loss Function

The loss function is a function to measure how well a model is performing on a given example. It takes the ground truth value of a variable ($y$) and the estimated value of that variable by the model ($\hat y$) which intuitively represents some kind of “cost” associated with the event. Our goal is to minimize the loss function.<br><br>
The loss function that we’ll be using to measure how good the neural network performs is one that you are already familiar with. The Log loss or Binary Cross Entropy. In fact this is the loss that most binary classifiers use. The Loss function for the $i^{th}$ training example is given by,<br>

$$L(y, \hat y) =  y \log (\hat y) + (1 - y) \log (1 - \hat y)$$

#### 1.2. How does this loss function work

In the loss function $y$ is the ground truth value that needs to be predicted and it takes on values 0 or 1 i.e, $y \in \{0, 1\}$ and $\hat y$ is obtained from forward propagation and takes on values between 0 and 1 i.e, $\hat y \in (0, 1)$. For a good classification model, $y$ and $\hat y$ should be close to each other.
<br><br>So,<br>
If $y = 1$, $L(y, \hat y) = - \log (\hat y)$. Therefore the closer $\hat y$ gets to $1$, $\log (\hat y)$ becomes less and less negative and hence the loss gets closer to 0.<br><br>
On the other hand, if $y = 0$, $L(y, \hat y) = - \log (1 - \hat y)$. Therefore the closer $\hat y$ gets to $0$, $\log (1 - \hat y)$ becomes less and less negative and hence the loss gets closer to 0.<br><br>
Also, if by some extremely small chance, $y = 1$ and $\hat y \approx 0$ or $y = 0$ and $\hat y \approx 1$, $L(y, \hat y) \rightarrow \infty$, which is what we would want for such a bad prediction.
<br><br>
<div class="alert alert-block alert-info">
    <b>Note</b>:<br>
    It might be helpful if you take a look at the log graph ($x \in (0,1)$) to see how exactly this works out.  
</div>

#### 1.3 The Cost Function

Since our training set contains $m$ training examples, we define the cost function as the average of loss function over the m training examples. In fact this cost function is what we will be actually minimizing. The cost function is given by,<br><br>
$$J(Y, \hat Y) = -\frac{1}{m} \sum\limits_{i = 1}^m y^{(i)} \log (\hat y^{(i)}) + (1 - y^{(i)}) \log (1 - \hat y^{(i)})$$
<br>
where, $y^{(i)}$ refers to the $i^{th}$ column of $Y$ i.e, ground truth value of $i^{th}$ training example.

#### 1.4. Vectorization

The above equation in vectorized format is given by,<br><br>
$$\boxed{J(Y, \hat Y) = -\frac{1}{m} * SUM(Y \log (\hat Y) + (1 - Y) \log (1 - \hat Y))}$$
<br>
Where,<br>
<ul>
    <li>$Y$ and $\hat Y$ are $1 \times m$ dimensional vectors.</li>
    <li>$ SUM $ refers to summing over all elements of the matrix.</li>
</ul>

<hr style="height:2px;border-width:2;color:gray">

## 2. Backpropagation

So now that you have measured how good or bad your output is using the cost function, how do you give feedback to your network about this  cost? How do you adjust your weights and biases to get a better output? Enter Backpropagation - the most mathematically intense part of a Neural Network.<br><br>
Backpropagation is a fancy name for the ‘chain rule’ which you would have definitely come across in your Calculus course (yup, MATH F111 it is). Eventhough backpropagation strictly refers to the computation of gradients of the loss function with respect to the different weights and biases in a Neural Network, it is generally used to refer to the entire learning algorithm of a Neural Network.<br><br>

<center><img src = "https://miro.medium.com/max/1400/1*YuotNxDwryjp3FiOwhVIkg.jpeg" width = "400"></center>
<br>
<div class = "alert alert-block alert-info">
<b>From here on, the Mathematics may seem complicated, but if you slowly go through it step by step, you can see that it's just simple calculus.</b>
</div>

#### 2.1. Overview

In backpropagation, our main goal is to find the gradient of the cost function $J$ with respect to the parameters of an $L$ layered Neural Network $W^{[1]}, b^{[1]}, W^{[2]}, b^{[2]}, \ldots W^{[L]}, b^{[L]}$ i.e, $\frac{\partial J}{\partial W^{[l]}}$ and $\frac{\partial J}{\partial b^{[l]}}$ for $l = 1, 2, \ldots L$. Once we can find these, we use it to update our parameters using our trusty algorithm Gradient Descent.

#### 2.2. Obtaining the Gradients

Consider a 3 layered Neural Network as shown<br>
<div class = "alert alert-block alert-info">
    Note that in the image:<br>
    <ul>
        <li>$Z1, A1, X1, \dots$ actually mean $Z_1, A_1, X_1 \dots$</li>
        <li>$W1, W2, b1, b2, \dots$ actually mean $W^{[1]}, W^{[2]}, b^{[1]}, b^{[2]}, \dots$</li>
        <li>$L1, L2 \dots$ refers to $Layer 1, Layer 2 \dots$
    </ul>
</div>
<img src = "https://ik.imagekit.io/cpj5jrovil/3layer_v6uGpBW26.JPG" width = "800"><br><br>

Let's see what we have and what we need.<br><br>

What we have:<br>
<b>Forward propagation equations in vectorized form</b><br>
<ul>
    <li>Layer 1: $Z^{[1]} = W^{[1]} . X + b^{[1]}$  and  $A^{[1]} = g^{[1]}(Z^{[1]})$</li>
    <li>Layer 2: $Z^{[2]} = W^{[2]} . A^{[1]} + b^{[2]}$  and  $A^{[2]} = g^{[2]}(Z^{[2]})$</li>
    <li>Layer 3: $Z^{[3]} = W^{[3]} . A^{[2]} + b^{[3]}$  and  $A^{[3]} = g^{[3]}(Z^{[3]}) = \hat Y$</li>
</ul><br>
where for the $l^{th}$ layer,<br>
<ul>
    <li>$W^{[l]}$ is the weight matrix with dimension $n^{[l]} \times n^{[l-1]}$.</li>
    <li>$b^{[l]}$ is the bias vector with dimension $n^{[l]} \times 1$.</li>
    <li>$g^{[l]}(.)$ is the activation function (e.g, sigmoid, relu, etc.).</li>
</ul>
<br>
<b>Cost function</b><br>
$J(Y, \hat Y) = -\frac{1}{m} \sum\limits_{i = 1}^m y^{(i)} \log (\hat y^{(i)}) + (1 - y^{(i)}) \log (1 - \hat y^{(i)})$
<br><br>
What we need:<br>
<b>Gradients of cost function with respect to parameters</b><br><br>
$$\frac{\partial J}{\partial W^{[3]}}, \frac{\partial J}{\partial b^{[3]}}, \frac{\partial J}{\partial W^{[2]}}, \frac{\partial J}{\partial b^{[2]}}, \frac{\partial J}{\partial W^{[1]}}, \frac{\partial J}{\partial b^{[1]}}$$
<br><br>
These equations can be represented by the following graph<br>
<center><img src = "https://ik.imagekit.io/cpj5jrovil/Forward_flow_KjdPF_OZJ.jpeg" width = "350"></center>
<br>

Seeing the above graph, we can get the gradients in the $3^{rd}$ layer as follows using chain rule:<br><br>
$$\begin{align}
\frac{\partial J}{\partial w^{[3]}} & = \frac{\partial J}{\partial a^{[3]}}.\frac{\partial a^{[3]}}{\partial z^{[3]}}.\frac{\partial z^{[3]}}{\partial w^{[3]}} \\\\
\frac{\partial J}{\partial b^{[3]}} & = \frac{\partial J}{\partial a^{[3]}}.\frac{\partial a^{[3]}}{\partial z^{[3]}}.\frac{\partial z^{[3]}}{\partial b^{[3]}}
\end{align}$$
<br><br>
For the $2^{nd}$ layer two more terms get added:<br><br>
$$\begin{align}
\frac{\partial J}{\partial w^{[2]}} & = \frac{\partial J}{\partial a^{[3]}}.\frac{\partial a^{[3]}}{\partial z^{[3]}}.\frac{\partial z^{[3]}}{\partial a^{[2]}}.\frac{\partial a^{[2]}}{\partial z^{[2]}}.\frac{\partial z^{[2]}}{\partial w^{[2]}} \\\\
\frac{\partial J}{\partial b^{[2]}} & = \frac{\partial J}{\partial a^{[3]}}.\frac{\partial a^{[3]}}{\partial z^{[3]}}.\frac{\partial z^{[3]}}{\partial a^{[2]}}.\frac{\partial a^{[2]}}{\partial z^{[2]}}.\frac{\partial z^{[2]}}{\partial b^{[2]}}
\end{align}$$
<br><br>
Similarly for the $1^{st}$ layer two more terms will get added.
<br><br>
These equations look very complicated. So we can use a computational graph like this to see how exactly we propagate backwards and what gradients we get in each step. Try comparing it with the above equations<br>
<center><img src = "https://ik.imagekit.io/cpj5jrovil/Backward_flow_731lDVkWT.jpeg" width = "250"></center>
<br>

Now let's compute the required gradients one by one. For now, we'll compute all the gradients for a single training example and generalize later on.<br><br>
<ol>
    <li>
        $\frac{\partial J}{\partial a^{[3]}}$:<br>
        $$\begin{align}
        \because \hat y = a^{[3]}, \qquad J &= - (y \log (a^{[3]}) + (1 - y) \log (1 - a^{[3]})) \\\\
        \implies \frac{\partial J}{\partial a^{[3]}} &= - \left(\frac{y}{a^{[3]}} - \frac{1-y}{1 - a^{[3]}}\right) \tag{1}
        \end{align}$$
    </li><br><br>
    <li>
        $\frac{\partial J}{\partial z^{[3]}}$:<br>
        $$\begin{align}
        \frac{\partial J}{\partial z^{[3]}} &= \frac{\partial J}{\partial a^{[3]}}.\frac{\partial a^{[3]}}{\partial z^{[3]}} \\\\
        \because a^{[3]} = g^{[3]}(z^{[3]}), \qquad \frac{\partial J}{\partial z^{[3]}} &= \underbrace{\frac{\partial J}{\partial a^{[3]}}}_{\text{(1)}}*g^{[3] \prime}(z^{[3]}) \tag{2}
        \end{align}$$<br>
        Notice that we use derivative of the activation function in this equation (which you would have seen in Neural Networks Part 1).
    </li><br><br>
    <li>
        $\frac{\partial J}{\partial w^{[3]}}$:<br>
        $$\begin{align}
        \frac{\partial J}{\partial w^{[3]}} &= \frac{\partial J}{\partial z^{[3]}}.\frac{\partial z^{[3]}}{\partial w^{[3]}} \\\\
        \because z^{[3]} = w^{[3]}.a^{[2]} + b^{[3]}, \qquad \frac{\partial J}{\partial w^{[3]}} &= \underbrace{\frac{\partial J}{\partial z^{[3]}}}_{\text{(2)}}.a^{[2]} \tag{3}
        \end{align}$$
    </li><br><br>
    <li>
        $\frac{\partial J}{\partial b^{[3]}}$:<br>
        $$\begin{align}
        \frac{\partial J}{\partial b^{[3]}} &= \frac{\partial J}{\partial z^{[3]}}.\frac{\partial z^{[3]}}{\partial b^{[3]}} \\\\
        \because z^{[3]} = w^{[3]}.a^{[2]} + b^{[3]}, \qquad \frac{\partial J}{\partial b^{[3]}} &= \underbrace{\frac{\partial J}{\partial z^{[3]}}}_{\text{(2)}}.(1) \tag{4}
        \end{align}$$
    </li><br><br>
    <li>
        $\frac{\partial J}{\partial a^{[2]}}$:<br>
        $$\begin{align}
        \frac{\partial J}{\partial a^{[2]}} &= \frac{\partial J}{\partial z^{[3]}}.\frac{\partial z^{[3]}}{\partial a^{[2]}} \\\\
        \because z^{[3]} = w^{[3]}.a^{[2]} + b^{[3]}, \qquad \frac{\partial J}{\partial a^{[2]}} &= \underbrace{\frac{\partial J}{\partial z^{[3]}}}_{\text{(2)}}.w^{[3]} \tag{5}
        \end{align}$$
    </li><br>
    This completes the computation for gradients involving 3rd layer. Let's compute some more gradients to see if we can notice any patterns.<br><br>
    <li>
        $\frac{\partial J}{\partial z^{[2]}}$:<br>
        $$\begin{align}
        \frac{\partial J}{\partial z^{[2]}} &= \frac{\partial J}{\partial a^{[2]}}.\frac{\partial a^{[2]}}{\partial z^{[2]}} \\\\
        \because a^{[2]} = g^{[2]}(z^{[2]}), \qquad \frac{\partial J}{\partial z^{[2]}} &= \underbrace{\frac{\partial J}{\partial a^{[2]}}}_{\text{(5)}}*g^{[2] \prime}(z^{[2]}) \tag{6}
        \end{align}$$<br><br>
        Compare this equation with (2)
    </li><br><br>
    <li>
        $\frac{\partial J}{\partial w^{[2]}}$:<br>
        $$\begin{align}
        \frac{\partial J}{\partial w^{[2]}} &= \frac{\partial J}{\partial z^{[2]}}.\frac{\partial z^{[2]}}{\partial w^{[2]}} \\\\
        \because z^{[2]} = w^{[2]}.a^{[1]} + b^{[2]}, \qquad \frac{\partial J}{\partial w^{[2]}} &= \underbrace{\frac{\partial J}{\partial z^{[2]}}}_{\text{(6)}}.a^{[1]} \tag{7}
        \end{align}$$
        Compare this equation with (3)
    </li><br><br>
    <li>
        $\frac{\partial J}{\partial b^{[2]}}$:<br>
        $$\begin{align}
        \frac{\partial J}{\partial b^{[2]}} &= \frac{\partial J}{\partial z^{[2]}}.\frac{\partial z^{[2]}}{\partial b^{[2]}} \\\\
        \because z^{[2]} = w^{[2]}.a^{[1]} + b^{[2]}, \qquad \frac{\partial J}{\partial b^{[2]}} &= \underbrace{\frac{\partial J}{\partial z^{[2]}}}_{\text{(6)}}.(1) \tag{8}
        \end{align}$$
        Compare this equation with (4)
    </li><br><br>
    <li>
        $\frac{\partial J}{\partial a^{[1]}}$:<br>
        $$\begin{align}
        \frac{\partial J}{\partial a^{[1]}} &= \frac{\partial J}{\partial z^{[2]}}.\frac{\partial z^{[2]}}{\partial a^{[1]}} \\\\
        \because z^{[2]} = w^{[2]}.a^{[1]} + b^{[2]}, \qquad \frac{\partial J}{\partial a^{[1]}} &= \underbrace{\frac{\partial J}{\partial z^{[2]}}}_{\text{(6)}}.w^{[2]} \tag{9}
        \end{align}$$
        Compare this equation with (5)
    </li>
</ol>
<br><br>

If you would have compared equations (6), (7), (8) and (9) with (2), (3), (4) and (5) respectively, you definitely would have noticed the similarity. In fact if you derive the equations for one more layer, the pattern repeats.<br>
From this we can infer the <b>four equations for backpropagation</b> as follows:

For any layer $l$ in a Neural Network,<br>
$$\begin{align}
1.& \; \frac{\partial J}{\partial z^{[l]}} = \frac{\partial J}{\partial a^{[l]}}*g^{[l] \prime}(z^{[l]}) \\\\
2.& \; \frac{\partial J}{\partial w^{[l]}} = \frac{\partial J}{\partial z^{[l]}}.a^{[l-1]} \\\\
3.& \; \frac{\partial J}{\partial b^{[l]}} = \frac{\partial J}{\partial z^{[l]}} \\\\
4.& \; \frac{\partial J}{\partial a^{[l-1]}} = \frac{\partial J}{\partial z^{[l]}}.w^{[l]}
\end{align}$$<br>
By applying these four equation for layers in the order $L, (L-1), \ldots 2, 1$ in an $L$ layered Neural Network you would have successfully implemented backpropagation.<br>
However there is one step that doesn't come under this pattern. Computation of $\frac{\partial J}{\partial a^{[L]}}$ where $L$ is the last layer. This needs to be computed first and separately as it involves directly differentiating the cost function. This can be seen in equation (1).<br><br>
<div class="alert alert-block alert-info"><b>
Note that there is no need to remember the entire math behind backpropagation. These four equations are what we'll be actually using. The math was given just for clarity and better understanding.</b>
</div>

#### 2.3. Vectorization

To increase computation speed we have already vectorized forward propagation and cost calculation. Likewise, let's vectorize backpropagation.

<div class="alert alert-block alert-info">
    <b>Notation information:</b>:<br>
    <ul>
        <li>
    From now on, to represent gradients we'll be using slightly different notations which although technically inaccurate will make writing and coding easier.<br>
    If we were to calculate the gradient of cost function $J$ with respect to a parameter $\theta$ i.e, $\frac{\partial J}{\partial \theta}$, we will represent it as $d \theta$.
    So $\frac{\partial J}{\partial w^{[l]}}$ will be represented as $dw^{[l]}$, $\frac{\partial J}{\partial A^{[l]}}$ will be represented as $dA^{[l]}$ and so on.<br>
        </li>
    <li>Uppercase character represents matrix form of the corresponding lowercase character. For example, $W^{[l]}$ is the matrix form of $w^{[l]}$.</li>
        <li>Also remember that in matrix form, <b>$\theta$ and $d\theta$ will have same shape</b>.</li>
    </ul>
</div>

For an L layered Neural Network (binary classification):
<ul>
    <li>$dA^{[L]}$ can be compuuted as:<br><br>
        $$\boxed{dA^{[L]} = - \left( \frac{Y}{A^{[L]}} - \frac{1-Y}{1 - A^{[L]}} \right)}$$
    </li><br>
    <li>The four equations of backpropagation for the $l^{th}$ are given by:<br><br>
        $$\boxed{\begin{align}
        1.& \, dZ^{[l]} = dA^{[l]} * g^{[l] \prime}(z^{[l]}) \\\\
        2.& \, dW^{[l]} = \frac{1}{m}dZ^{[l]}.A^{[l-1]^T} \\\\
        3.& \, db^{[l]} = \frac{1}{m} \sum _{col} dZ^{[l]} \\\\
        4.& \, dA^{[l-1]} = W^{[l]^T}.dZ^{[l]}
        \end{align}}$$<br>
        where,<br>
        <ul>
            <li>$\sum \limits_{col}$ refers to <b>summing across columns</b>.</li>
            <li>$m$ is the number of training examples.</li>
        </ul>
</ul>
<div class="alert alert-block alert-info">
    <b>Some important point to note:</b>
    <ul>
        <li>Notice how the formula changes slightly for matrix form from elementwise form.</li>
        <li>Don't forget that we use the derivative of the activation function in equation 1</li>
        <li>'$.$' refers to matrix multiplication while '$*$' refers to elementwise multiplication.</li>
    </ul>
</div>

#### 2.4. Updating Parameters

Once the gradients for parameters of all layers have been calculated we proceed to update the parameters using Gradient Descent like we did for Linear and Logistic Regressions.

If we perform gradient descent for $T$ iterations for a Neural Network of $L$ layers:<br><br>
<center>
$\boxed{\begin{align}
for \; &t = 1 \ldots T: \\\\
&for \; l = 1 \ldots L: \\\\
&\qquad W^{[l]} = W^{[l]} - \alpha*dW^{[l]} \\\\
&\quad \quad \ \ \ b^{[l]} = b^{[l]} - \alpha*dW^{[l]}
\end{align}}$
</center>

<hr style="height:2px;border-width:2;color:gray">

## 3. Mini-batch Gradient Descent

While working with Neural Networks, we generally deal with large amount of data. Suppose you are dealing with a dataset having 500,000 training examples. Then during forward propagation, how many examples do you need to process to perform a single step of gradient descent? All 500,000 of them, since we are taking it as a single matrix. As the Neural Network becomes deeper (i.e, th nuumber of layers increase) the computation becomes more and more expensive. So mini-batch gradient descent divides the training set into smaller 'mini-batches' and processes one mini-batch for a single gradient descent step. Let's look into this in a little more detail.<br>

Consider our training set with 500,000 examples. Now let's divide this training set into mini-batches of 1000 examples. To do this we first randomly shuffle the training data i.e, columns in $X$ and <b>corresponding</b> columns in $Y$. Once this is done, we take the first 1000 examples i.e, the first 1000 columns of $X$ and <b>corresponding</b> columns in $Y$ (remember that in $X$ and $Y$ columns refer to training examples) and call it mini-batch $1$ i.e, $( X^{\{1\}}, Y^{\{1\}})$. We take the next 1000 examples similarly and call it mini-batch $2$ i.e, $( X^{\{2\}}, Y^{\{2\}})$ and so on. In the end, for 500,000 examples we'll have 500 mini-batches with 1000 examples each.<br>

Now we take each mini-batch instead of the entire training set and perform one step of gradient descent on it. So for example, our forward propagation equation which was $Z^{[1]} = W^{[1]}.X + b^{[1]}$ will become, $Z^{[1]} = W^{[1]}.X^{\{t\}} + b^{[1]}$ for the $t^{th}$ mini-batch. We also use this mini-batch to perform one step of gradient descent. So by the time we use the entire training set once, we would have performed 500 steps of gradient descent, where on the other hand in the previous algorithm one pass through the training set would perform a single step of gradient descent.<br>

One pass through the entire training set i.e, one iteration over all the mini-batches is called 1 <b>epoch</b>. So we can have multiple epochs to perform multiple passes through the training set.

Let's look at a side by side comparison of the two methods.
<table>
    <tr>
        <th>Gradient Descent</th>
        <th>Mini-batch Gradient Descent</th>
    </tr>
    <tr>
        <td>
            $$\begin{align}
            for \; &i = 1 \ldots num\_iterations: \\\\
            & \{Gradient \, Descent \, Equations \, for \, X \, and \, Y\}
            \end{align}$$
        </td>
        <td>
            $$\begin{align}
            for \; &i = 1 \ldots num\_epochs: \\\\
            &for \; t = 1 \ldots num\_minibatches: \\\\
            & \qquad \{Gradient \, Descent \, Equations \, for \, X^{\{t\}} \, and \, Y^{\{t\}} \}
            \end{align}$$
        </td>
    </tr>
</table>

To understand why this algorithm works better for large datasets and how it differs from the one we have been seeing so far, make sure you check out the videos linked in the Additional Resources section.

<hr style="height:2px;border-width:2;color:gray">

## 4. Multiclass Classification

#### 4.1. Introduction

So far we've looked at a Neural Network which can perform binary classifications i.e, 0 or 1. However many times we need to deal with multiclass classification tasks like, recognizing handwritten digits (0-9), Iris data (classify into 3 flowers), etc. It's very simple to transition from binary classification to multiclass classification. The only difference is in the activation function of the output layer. While we use the <b>sigmoid</b> activation in binary classification, we'll use the <b>softmax</b> activation function for multiclass classification.

#### 4.2. The Softmax Activation

The softmax activation function is given by, <br>
$$\boxed{S(z_i) = \frac{e^{z_i}}{\sum\limits_{j=1}^{n^{[L]}} e^{z_j}}}$$
<br><br>
In the ouput layer you will have computed $Z^{[L]}$, an $(n^{[L]} \times 1)$ vector. The softmax function takes this entire vector as input and ouputs another vector $A^{[L]}$ of same dimensions where each entry is a number between 0 and 1. Also note that sum off all values in $A^{[L]}$ will be equal to 1.

#### 4.3. Cost function

Since our activation function changes at the output layer we will need to modify the Cost function. The new cost function for the multiclass classifier is similar to the old one and is called <b>Categorical Cross Entropy</b> and defined by,<br><br>
$$\boxed{J(Y, \hat Y) = - \frac{1}{m} SUM(Y \log (\hat Y))}$$
<br>
Note that here $Y$ and $\hat Y = A^{[L]}$ will be $(n^{[l]} \times m)$ shaped matrices in their one hot representations. Also since the cost functions change, the first step of backpropagation will also face minor changes. Try to figure out those changes.

<div class="alert alert-block alert-info">
    <b>Note</b>:
    There is also one more way to perform multiclass classification. It is called the <b>One vs All</b> classifier. If we have 10 classes, we train 10 binary classifiers where each one classifies one  particular class . Try finding out how exactly this method works.
</div>

<hr style="height:2px;border-width:2;color:gray">

## 5. Conclusion

So now that we have looked at Neural Networks in detail, let's note all the important steps to remember. This will help you fit all the pieces together.

#### 5.1. Neural Network Workflow (for a Binary Classifier)

<ol>
    <li>
        <b>Defining the Neural Network Architecture</b>: Define the structure of your Neural Network by chosing the number of layers, number of units in each layer and the activation function of each layer.
    </li>
    <li>
        <b>Initializing parameters</b>: Initialize your Weight matrices randomly and Bias vectors to zeros. Make sure your matrices and vectors are in their <b>correct shapes</b>.
    </li>
    <li>
        <b>Mini-batch creation</b>: Divide your traing data $X_{train}$ and $Y_{train}$ into proper mini-batches.
    </li>
    <li>
        <b>Training</b>:<br>
        Now you start your training process. Iterate for a certain number of <b>epochs</b> and for each epoch iterate through all the mini-batches. For each mini-batch perform the following steps:<br>
        <ul>
            <li><b>Forward Propagation</b>: Propagate through the Neural network for layers $l = 1 \ldots L$. The equations for the $l^{th}$ layer are:<br><br>
                $$\boxed{\begin{align}
                Z^{[l]} &= W^{[l]}.A^{[l-1]} + b^{[l]} \\\\
                A^{[l]} &= g^{[l]}(Z^{[l]})
                \end{align}}$$<br>
            </li>
            <li><b>Compute the Cost</b>: Compute the cost for this iteration using,<br><br>
                $$\boxed{J(Y, \hat Y) = -\frac{1}{m} * SUM(Y \log (\hat Y) + (1 - Y) \log (1 - \hat Y))}$$<br>
                where, $\hat Y = A^{[L]}$
            </li>
            <li>
                <b>Backpropagation</b>:<br>
                <ul>
                    <li><b>Compute gradient at output layer</b> using,<br><br>
                        $$\boxed{dA^{[L]} = - \left( \frac{Y}{A^{[L]}} - \frac{1-Y}{1 - A^{[L]}} \right)}$$<br>
                    </li>
                    <li><b>Apply the four equations of backrpopagation</b> to compute gradients through layers $l = L \ldots 1$. The equations for $l^{th}$ layer are,<br><br>
                         $$\boxed{\begin{align}
                            1.& \, dZ^{[l]} = dA^{[l]} * g^{[l] \prime}(z^{[l]}) \\\\
                            2.& \, dW^{[l]} = \frac{1}{m}dZ^{[l]}.A^{[l-1]^T} \\\\
                            3.& \, db^{[l]} = \frac{1}{m} \sum _{col} dZ^{[l]} \\\\
                            4.& \, dA^{[l-1]} = W^{[l]^T}.dZ^{[l]}
                            \end{align}}$$<br>
                    </li>
                </ul>
            </li>
            <li><b>Update the parameters</b>: Using the gradients computed through backpropagation, update the Weights and Biases for layers $l = 1 \ldots L$ using,<br><br>
                $$\boxed{\begin{align}
                W^{[l]} & = W^{[l]} - \alpha * dW^{[l]} \\
                b^{[l]} & = b^{[l]} - \alpha * db^{[l]}
                \end{align}}$$<br>
                where, $\alpha$ is the learning rate.
            </li>
        </ul>
        This completes the training process.
    </li>
    <li>
        <b>Evaluation</b>: Evaluate your model using $X_{test}$ and $Y_{test}$. Remember that we want our model to generalize to unseen data. So it is the $test$ set accuracy that actually matters.
    </li>
</ol>

<b>By implementing these 5 steps correctly you will have successfully implemented an Articial Neural Network.</b>

#### 5.2. Summary of notations and shapes

<ul>
    <li>
        <h5><b>Notations</b></h5>
        <ol>
            <li>
                $X^{(i)\{t\}[l]}_j$ says that the quantity $X$,<br>
                <ul>
                    <li>Belongs to the $[l]^{th}$ layer.</li>
                    <li>Belongs to the $\{t\}^{th}$ mini-batch.</li>
                    <li>Is part of the $(i)^{th}$ training example.</li>
                    <li>Is located at the $j^{th}$ unit of that layer.</li>
                </ul>
            </li>
            <li>$n^{[l]}$ denotes the number of units in the $l^{th}$ layer.</li>
            <li>$m$ denotes the number of training examples.</li>
        </ol>
    </li>
    <li>
        <h5><b>Shapes</b></h5>
        <center>
            <table>
                <tr>
                    <th>Quantity</th>
                    <th>Shape</th>
                </tr>
                <tr>
                    <td>$$X$$</td>
                    <td>$$n_x \times m$$</td>
                </tr>
                <tr>
                    <td>$$Y (binary)$$</td>
                    <td>$$1 \times m$$</td>
                </tr>
                <tr>
                    <td>$$Y (multiclass)$$</td>
                    <td>$$n^{[L]} \times m$$</td>
                </tr>
                <tr>
                    <td>$$Z{[l]}$$</td>
                    <td>$$n^{[l]} \times m$$</td>
                </tr>
                <tr>
                    <td>$$A{[l]}$$</td>
                    <td>$$n^{[l]} \times m$$</td>
                </tr>
                <tr>
                    <td>$$W^{[l]}$$</td>
                    <td>$$n^{[l]} \times n^{[l-1]}$$</td>
                </tr>
                <tr>
                    <td>$$b^{[l]}$$</td>
                    <td>$$n^{[l]} \times 1$$</td>
                </tr>
            </table>
        </center>
    </li>
</ul>

#### 5.3. Hyperparameters

While the Weights and Biases are <b>parameters</b> that your model learns on its own, there are some parameters which it can't learn. These parameters are called <b>hyperparameters</b> and are manually set by the ML engineer. You've already come across many hyperparameters. The number of layers, The number of units in each layer, Activation functions, Learning rate, Mini-batch size, Number of epochs are all hyperparameters. By tuning the hyperparameters, you can improve the performance of the model. In fact, if you've read the "Introduction to ML" article, you would have seen a step called "Cross-Validation". That step is generally used for hyperparameter tuning.

#### 5.4. Deep Learning

Deep Learning is a sub-field of Machine Learning that deals with Supervised and Unsupervised Learning using Networks. Artificial Neural Networks (ANNs) which you have just learnt is the first step in Deep Learning. Other Neural Networks include, Convolutional Neural Networks (CNNs) which are used for tasks like image recognition; Recurrent Neural Networks (RNNs) which are used for Sequence Models like Speech Recognition, Time Series Analysis, Natural Language Processing etc. Deep Learning is a vast field and has lots of applications in the real world.

<hr style="height:2px;border-width:2;color:gray">

### Additional Resources

<ul>
    <li>For an amazing intuition and clarity of how Neural Networks work, check out <a href = "https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi">this</a> playlist by 3 Blue 1 Brown. Even though it takes around 1 hour, it's definitely worth it.</li>
    <li>Needless to say, Andrew Ng's Deep Learning Specialization is one of the best, if you want to go deeper into this field of Machine Learning. <a href = "https://www.youtube.com/playlist?list=PLkDaE6sCZn6Ec-XTbcX1uRg2_u4xOEky0&fbclid=IwAR2W10I3Enkrp15J7Eb2w-X-F9BwvVnwgGObAGx-Swz7SEektkiPMlDzufw">Here</a> is the playlist dealing with ANNs.</li>
    <li>For a better understanding of Backpropagation check <a href = "https://medium.com/binaryandmore/beginners-guide-to-deriving-and-implementing-backpropagation-e3c1a5a1e536">this</a> article.</li>
    <li>Mini-batch Gradient Descent: <a href = "https://youtu.be/4qJaSmvhxi8">Part 1</a> and <a href = "https://youtu.be/-_4Zi8fCZO4">Part 2</a>.</li>
    <li>Softmax: <a href = "https://youtu.be/LLux1SW--oM">Part 1</a> and <a href = "https://youtu.be/ueO_Ph0Pyqk">Part 2</a>.</li>
</ul>

### References

<ul>
    <li>The notations used in this notebook are greatly influenced by Andrew Ng's Deep Learning Specialization.</li>
    <li><a href = "https://medium.com/binaryandmore/beginners-guide-to-deriving-and-implementing-backpropagation-e3c1a5a1e536">This</a> Medium Article was used as reference for Backpropagation. The Forward popagation computation graph, Backward propagation computation graph and the meme were taken from the same article.</li>
</ul>

<hr style="height:5px;border-width:2;color:gray">